In [1]:
from fastapi import FastAPI, HTTPException
import httpx
from pydantic import BaseModel

app = FastAPI()

# Configuración de la API de Moodle
MOODLE_URL = "http://localhost/moodle/webservice/rest/server.php"
MOODLE_TOKEN = "7f0aa2b0746edf87b136e976cfaac531"


class MessageData(BaseModel):
    touserid: int
    text: str
    textformat: int
    


# Ruta para actualziar un curso en Moodle utilizando el método POST
@app.post("/send_message_moodle")
# Función para actualizar un curso existente en Moodle
async def send_message_moodle( message_data: MessageData):
    # Define la función de la API de Moodle para actualizar cursos
    moodle_api_function = "core_message_send_instant_messages"

    # Configura los parámetros para la solicitud
    params = {
        "wstoken": MOODLE_TOKEN,
        "wsfunction": moodle_api_function,
        "moodlewsrestformat": "json",
       # "courses[0][id]": course_data.id,  # ID del curso existente
        
        "messages[0][touserid]": message_data.touserid,
        "messages[0][text]": message_data.text,
        "messages[0][textformat]": message_data.textformat,

    }

    # Realiza la solicitud a la API de Moodle
    async with httpx.AsyncClient() as client:
        response = await client.post(MOODLE_URL, params=params)

    # Verifica el estado de la respuesta
    if response.status_code == 200:
        result = response.json()
        if "exception" in result:
            raise HTTPException(status_code=500, detail=f"Error en la API de Moodle: {result['exception']}")
        else:
            return {"message": "Mensaje enviado exitosamente"}
    else:
        raise HTTPException(status_code=500, detail=f"Error al llamar a la API de Moodle: {response.text}")








In [2]:
#Poner en marcha el servidor en un hilo 
import nest_asyncio
nest_asyncio.apply()

from threading import Thread
import uvicorn

# Variable de bandera para controlar el hilo
stop_server = False

def run_server():
    uvicorn.run(app, host="127.0.0.1", port=8000)

# Inicia el servidor en un hilo
server_thread = Thread(target=run_server)
server_thread.start()



INFO:     Started server process [16839]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


In [1]:
#Probar las llamadas http a la api de fastapi desde 
import json
import httpx

# Lee los datos del usuario desde el archivo JSON
with open("data/message_data.json", "r") as file:
    message_data_json = json.load(file)

# Realiza una solicitud POST a la ruta /add_user_moodle
url = "http://localhost:8000/send_message_moodle"
response = httpx.post(url, json=message_data_json)

# Imprime la respuesta del servidor
print(response.text)

Internal Server Error
